# Классификаиция клиентов телеком компании

## Содержание
1. [Описание проекта](#id_description)
2. [Данные](#id_start)
3. [Обучение моделей](#id_learning)
4. [Выбор модели](#id_choise)
5. [Проверка модели на вменяемость](#id_check)
6. [Общий вывод](#id_conclusion)

## Описание проекта
<a id="id_description"></a>

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

**Цель**

Построить модель для задачи классификации, которая выберет подходящий тариф для клиентов

**Инструкция по выполнению проекта**

   - Откройте файл с данными и изучите его. Путь к файлу: `datasets/users_behavior.csv`.
   - Разделите исходные данные на обучающую, валидационную и тестовую выборки.
   - Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
   - Проверьте качество модели на тестовой выборке.
   - Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score

## Данные
<a id="id_start"></a>

In [2]:
# загрузка данных
try:
    data_users = pd.read_csv('data-files/users_behavior.csv')  # Локальный путь
except:
    data_users = pd.read_csv('/datasets/users_behavior.csv')  # Серверный путь

In [3]:
data_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data_users.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод

Имеется набор данных о клиенте компании «Мегалайн». 
В данных есть информация о поведении одного пользователя за месяц:
количесво звонков и их суммарная длительность, количество сообщений, израсходованный интернет-трафик, и тариф, которым пользовался клиент («Ультра» или «Смарт»).

Данные уже подготовлены для дальнейшей работы и не требуют предобработки.

## Обучение моделей
<a id="id_learning"></a>

Разобьем исходные данные на обучающую, валидационную и тестовую выборки в пропорции 3:1:1.

In [5]:
# выделим тестовую выборку
df_train, df_test = train_test_split(data_users, test_size=0.2, random_state=12345)

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# выделим обучающую и валидационную выборки
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [6]:
# Проверка на соответсвие размеров выборок
print('Размер тестовой выборки:')
print('features_test:',features_test.shape)
print('target_test:',target_test.shape)

print()
print('Размер валидационной выборки:')
print('features_valid:',features_valid.shape)
print('target_valid:',target_valid.shape)

print()
print('Размер обучающей выборки:')
print('features_train:',features_train.shape)
print('target_train:',target_train.shape)

Размер тестовой выборки:
features_test: (643, 4)
target_test: (643,)

Размер валидационной выборки:
features_valid: (643, 4)
target_valid: (643,)

Размер обучающей выборки:
features_train: (1928, 4)
target_train: (1928,)


Выберем лучшие параметры для моделей типа "дерево решений" и "случайный лес". А также сравним accuracy для моделей "логическая регрессия", "дерево решений" и "случайный лес" при лучших параметрах.

In [7]:
# DecisionTreeClassifier
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
for depth in range(1, 6):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train,target_train)
    predictions_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_tree)
    print('max_depth =',depth,
          '\taccuracy train set =', accuracy_score(target_train, model_tree.predict(features_train)),
          '\taccuracy valid set =', accuracy_score(target_valid, predictions_tree))
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result_tree
        best_depth_tree = depth

max_depth = 1 	accuracy train set = 0.758298755186722 	accuracy valid set = 0.7387247278382582
max_depth = 2 	accuracy train set = 0.79201244813278 	accuracy valid set = 0.7573872472783826
max_depth = 3 	accuracy train set = 0.8117219917012448 	accuracy valid set = 0.7651632970451011
max_depth = 4 	accuracy train set = 0.8205394190871369 	accuracy valid set = 0.7636080870917574
max_depth = 5 	accuracy train set = 0.8272821576763485 	accuracy valid set = 0.7589424572317263


In [8]:
# RandomForestClassifier
best_model_forest = None
best_result_forest = 0
best_est_forest = 0
best_depth_forest = 0
for est in range(1, 11):
#    print('n_estimators =',est)
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth=depth)
        model_forest.fit(features_train,target_train)
        predictions_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(target_valid, predictions_forest)
#        print('max_depth =',depth,
#          '\taccuracy train set =', accuracy_score(target_train, model_forest.predict(features_train)),
#          '\taccuracy valid set =', accuracy_score(target_valid, predictions_forest))
        if result_forest > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result_forest
            best_est_forest = est
            best_depth_forest = depth

In [9]:
# LogisticRegression
model_log = LogisticRegression(random_state=12345)
model_log.fit(features_train,target_train)
predictions_log = model_log.predict(features_valid)
result_log = accuracy_score(target_valid, predictions_log)

In [10]:
print("Accuracy моделей на валидационной выборке:")
print("\"дерево решений\":\t", best_result_tree, "Глубина дерева:\t", best_depth_tree)
print("\"случайный лес\":\t", best_result_forest, 
      "Количество деревьев:", best_est_forest, "Максимальная глубина:\t", best_depth_forest)
print("\"логическая регрессия\":\t", result_log)

Accuracy моделей на валидационной выборке:
"дерево решений":	 0.7651632970451011 Глубина дерева:	 3
"случайный лес":	 0.7962674961119751 Количество деревьев: 3 Максимальная глубина:	 9
"логическая регрессия":	 0.7262830482115086


## Тестирование модели
<a id="id_choise"></a>

Наибольшее accuracy для валидационной выборки получилось у модели "случайного леса" с гиперпараметрами `количество деревьев` = 3, `максимальная глубина` = 9.
Применим ее для тестовой выборки

In [11]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=best_est_forest,max_depth=best_depth_forest)
model_forest.fit(features_train,target_train)
predictions_forest = model_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_forest)
print("Accuracy случайного леса на тестовой выборке:", result_forest)

Accuracy случайного леса на тестовой выборке: 0.7931570762052877


## Проверка модели на вменяемость
<a id="id_check"></a>

Проверку модели на вменяемтость проведем путем сравнения выбранной модели с dummy-классификатором. 
Dummy-классификатор используется как basesline, по сравнению с которой другие модели должны давать лучший результат.
В dummy классификатор будем использовать с гиперпараметром `strategy`='most_frequent'.

In [12]:
dummy_clf = DummyClassifier(strategy='most_frequent',random_state=12345)
dummy_clf.fit(features_train,target_train)
predictions_dummy = dummy_clf.predict(features_test)
result_dummy = accuracy_score(target_test, predictions_dummy)
print("Accuracy dummy-классификатора на тестовой выборке:", result_dummy)

Accuracy dummy-классификатора на тестовой выборке: 0.6951788491446346


## Общий вывод
<a id="id_conclusion"></a>

В работе были исследовали "логическая регрессия", "дерево решений" и "случайный лес"


Наилучшую эффективность показала модель "случайный лес" с гиперпараметрами с гиперпараметрами `количество деревьев` = 3, `максимальная глубина` = 9.

Модель "случайный лес" с наилучшими гиперапараметрами была проверена на тестовой выборке и показала результат Accuracy = 0.793.

Также модель была проверена на вменяемость.
Accuracy dummy-классификатора = 0.695.
Выбранная модель обладает большей предсказатьельной точностью.